In [13]:
pip install pytorch_lightning

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets
from torchvision.models import resnet50, ResNet50_Weights
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import StratifiedShuffleSplit


In [16]:
# -------------------------
# 1. Data and Transforms
# -------------------------
DATA_DIR = "/content/drive/MyDrive/nature_12K/inaturalist_12K"
IMG_SIZE = 224      # Standard size for ResNet
BATCH_SIZE = 32
NUM_CLASSES = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Standard ImageNet normalization
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load full train folder (no transform) to get labels for stratification
full_train = datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform=None)
labels = [label for _, label in full_train.samples]

# Stratified split: 80% train, 20% validation
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(labels)), labels))

# Create Subsets with appropriate transforms
train_dataset = Subset(
    datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform=train_transforms),
    train_idx
)
val_dataset = Subset(
    datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform=val_transforms),
    val_idx
)

# Test dataset
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'), transform=val_transforms)

workers=2
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=workers, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=workers, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=workers, pin_memory=True)

# -------------------------
# 2. Model Setup
# -------------------------
# Load pretrained ResNet50 using new weights API
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
in_feats = model.fc.in_features
model.fc = nn.Linear(in_feats, NUM_CLASSES)
model = model.to(device)

# -------------------------
# 3. Freezing Strategies
# -------------------------
def freeze_all_but_fc(model):
    for name, param in model.named_parameters():
        if 'fc' not in name:
            param.requires_grad = False

def freeze_up_to_layer(model, layer_name):
    freeze = True
    for name, param in model.named_parameters():
        if freeze:
            param.requires_grad = False
        if layer_name in name:
            freeze = False

def freeze_first_two_blocks(model):
    for name, param in model.named_parameters():
        if 'layer1' in name or 'layer2' in name:
            param.requires_grad = False

# Choose a strategy (uncomment one):
# Strategy 1: Freeze all except FC
# freeze_all_but_fc(model)
Strategy="freeze_all_but_fc"

# Strategy 2: Freeze up to layer3
freeze_up_to_layer(model, 'layer3')
Strategy="freeze_up_to_layer"

# Strategy 3: Freeze first two blocks (layer1 and layer2)
# freeze_first_two_blocks(model)
Strategy="freeze_first_two_blocks"

# -------------------------
# 4. Lightning Module
# -------------------------
class FineTuneModule(pl.LightningModule):
    def __init__(self, model, lr=1e-4):
        super().__init__()
        self.model = model
        self.lr = lr
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss_resnet50', loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log('train_acc_resnet50', acc, prog_bar=True, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss_resnet50', loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log('val_acc_resnet50', acc, prog_bar=True, on_epoch=True, on_step=False)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss_resnet50', loss, prog_bar=True)
        self.log('test_acc_resnet50', acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.model.parameters()), lr=self.lr)
        return optimizer


In [17]:
# -------------------------
# Evaluation
# -------------------------
def evaluate_model(model, dataloader, device):
    model.to(device)  # Ensure model is on the correct device
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total


In [18]:
wandb_logger = WandbLogger(project="CNN_inaturalist_12K", name=f"fine_tune_resnet50_{Strategy}")
module = FineTuneModule(model, lr=1e-5)
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    precision=16 if torch.cuda.is_available() else 32,
    logger=wandb_logger,
    log_every_n_steps=10
)

# Train and validate
trainer.fit(module, train_loader, val_loader)

/usr/local/lib/python3.11/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/pytho

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [19]:
# Test evaluation: logs test_acc and test_loss to wandb
trainer.test(module, dataloaders=test_loader)

# Manual evaluation and printing
train_acc = evaluate_model(model, train_loader, device)
val_acc   = evaluate_model(model, val_loader, device)
test_acc  = evaluate_model(model, test_loader, device)
print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Validation Accuracy: {val_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Save model weights
torch.save(model.state_dict(), 'fine_tuned_resnet50.pth')

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_acc_resnet50     │    0.8840000033378601     │
│    test_loss_resnet50     │    0.3907071650028229     │
└───────────────────────────┴───────────────────────────┘

Train Accuracy: 97.95%
Validation Accuracy: 87.90%
Test Accuracy: 88.40%
